In [1]:
# Import Libaries and Tools
import os
import sys
from astropy.io import fits
import sunpy.map
from aiapy.calibrate import register, update_pointing
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.gridspec import GridSpec
import imageio
import glob

# ACWE utilities
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import ACWE Tools
sys.path.append(ROOT_DIR)
from ACWE_python_spring_2023 import acweSaveSeg_v5, acweRestoreScale
from DatasetTools import DataManagmentTools as dmt

In [2]:
# File
dataset = '/home/jgra/Coronal Holes/newDataset/'
ImageNames = ['CR2099/2010-07-25T120002Z/aia.lev1_euv_12s.2010-07-25T120009Z.193.image_lev1.fits',
              'CR2099/2010-08-06T080002Z/aia.lev1_euv_12s.2010-08-06T080009Z.193.image_lev1.fits',
              'CR2099/2010-08-06T160002Z/aia.lev1_euv_12s.2010-08-06T160009Z.193.image_lev1.fits',
              'CR2100/2010-08-21T220002Z/aia.lev1_euv_12s.2010-08-21T220009Z.193.image_lev1.fits',
              'CR2101/2010-09-18T000002Z/aia.lev1_euv_12s.2010-09-18T000008Z.193.image_lev1.fits',
              'CR2133/2013-02-21T230001Z/aia.lev1_euv_12s.2013-02-21T230008Z.193.image_lev1.fits']

specialCase = 'aia.lev1_euv_12s.2010-08-06T050009Z.193.image_lev1.fits'

# ACWE Folders
oldConMapFolder = '/mnt/coronal_holes/Code Paper I Observations/ConMapStandardOld/'
newConMapFolder = '/mnt/coronal_holes/Code Paper I Observations/ConMapStandardDefault/'

In [3]:
# Open File and Prepare for ACWE
def openAIA(filename):
    
    # Extract Image and Header Data
    hdulist = fits.open(filename)
    hdulist.verify('silentfix') # no clue why this is needed for successful data read
    h = hdulist[1].header
    J = hdulist[1].data
    hdulist.close()
    
    # Update to Level 1.5 Data Product
    if h['LVL_NUM'] < 1.5:
        m = sunpy.map.Map((J,h))    # Create Sunpy Map
        m = update_pointing(m)      # Update Header based on Latest Information
        m_registrered = register(m) # Recenter and rotate to Solar North
        I = m_registrered.data
        # Undo Keword Renaming
        H = dict()
        for k in m_registrered.meta.keys(): 
            H[k.upper()] = m_registrered.meta[k] 
    # Skip if already Level 1.5
    else:
        I = J*1
        H = dict(h)
        
        # Prepare Display Version
    Idsp = np.clip(I,20,2500)
    Idsp = np.log10(Idsp)
    Idsp = Idsp - np.min(Idsp)
    Idsp = Idsp/np.max(Idsp)
    
    return I,Idsp,H

In [4]:
# Ensure Folder Exists
saveDirectory = '/Figures/ConMaps/OldMaps/'
saveDirectory = ROOT_DIR + saveDirectory

if not os.path.exists(saveDirectory):
    os.makedirs(saveDirectory)

In [ ]:
for ImageName in ImageNames:
    
    # Get File Names
    filename = dataset + ImageName
    ACWEfiles = ImageName.split('/')[0]+'/'+ImageName.split('/')[1]+'/'
    oldConMap = glob.glob(oldConMapFolder + ACWEfiles + '*' + os.path.basename(ImageName) + '.npz')[0]
    
    # Get File Times
    time = dmt.timeFormat(dmt.timeFromFilenameAIA(ImageName))
    
    # Open AIA Image
    I,Idsp,H = openAIA(filename)
    
    # Open Old Confidence Maps
    H,AH,SEG = acweSaveSeg_v5.openSeg(oldConMap)
    
    # Open and save each mask
    fig=plt.figure(layout="constrained",figsize=[30,20])
    gs = GridSpec(4,6,figure=fig)
    plt.rcParams.update({'font.size': 60})
    fig.suptitle(time)
    plt.rcParams.update({'font.size': 50})
    for i in range(5):
        
        # Extract File
        if i == 0:
            j = np.where(AH['BACKGROUND_WEIGHT']==1/10.)[0][0]
            seg = SEG[j]
            seg = acweRestoreScale.upscale(seg,AH)
            ax1 = fig.add_subplot(gs[0:2,0:2])
            ax1.imshow(np.flip(Idsp,axis=0),cmap ='gray')
            ax1.contour(np.flip(seg,axis=0),0,colors='r')
            ax1.axis(False)
            ax1.set_title(r'$\lambda_i/\lambda_o=10$')
        elif i == 1:
            j = np.where(AH['BACKGROUND_WEIGHT']==1/25.)[0][0]
            seg = SEG[j]
            seg = acweRestoreScale.upscale(seg,AH)
            ax2 = fig.add_subplot(gs[0:2,2:4])
            ax2.imshow(np.flip(Idsp,axis=0),cmap ='gray')
            ax2.contour(np.flip(seg,axis=0),0,colors='r')
            ax2.axis(False)
            ax2.set_title(r'$\lambda_i/\lambda_o=25$')
        elif i == 2:
            j = np.where(AH['BACKGROUND_WEIGHT']==1/50.)[0][0]
            seg = SEG[j]
            seg = acweRestoreScale.upscale(seg,AH)
            ax3 = fig.add_subplot(gs[0:2,4:6])
            ax3.imshow(np.flip(Idsp,axis=0),cmap ='gray')
            ax3.contour(np.flip(seg,axis=0),0,colors='r')
            ax3.axis(False)
            ax3.set_title(r'$\lambda_i/\lambda_o=50$')
        elif i == 3:
            j = np.where(AH['BACKGROUND_WEIGHT']==1/75.)[0][0]
            seg = SEG[j]
            seg = acweRestoreScale.upscale(seg,AH)
            j = np.where(AH['BACKGROUND_WEIGHT']==1/50.)[0][0]
            seg = SEG[j]
            seg = acweRestoreScale.upscale(seg,AH)
            ax4 = fig.add_subplot(gs[2:4,0:4])
            ax4.imshow(np.flip(Idsp,axis=0),cmap ='gray')
            ax4.contour(np.flip(seg,axis=0),0,colors='r')
            ax4.axis(False)
            ax4.set_title(r'$\lambda_i/\lambda_o=75$')
        else:
            j = np.where(AH['BACKGROUND_WEIGHT']==1/100.)[0][0]
            seg = SEG[j]
            seg = acweRestoreScale.upscale(seg,AH)
            ax5 = fig.add_subplot(gs[2:4,3:5])
            ax5.imshow(np.flip(Idsp,axis=0),cmap ='gray')
            ax5.contour(np.flip(seg,axis=0),0,colors='r')
            ax5.axis(False)
            ax5.set_title(r'$\lambda_i/\lambda_o=100$')
    file = saveDirectory + 'samples.' + os.path.basename(ImageName) + '.png'
    plt.savefig(file)
    fig.show()
    
    # Other Figures
    plt.rcParams.update({'font.size': 60})
    plt.figure(figsize=[55,20])
    plt.suptitle(time)
    plt.rcParams.update({'font.size': 50})
    plt.subplot(1,2,1)
    plt.imshow(np.flip(Idsp,axis=0),cmap ='gray')
    plt.axis(False)
    plt.title('Observation')
    plt.subplot(1,2,2)
    plt.imshow(np.flip(np.sum(SEG,axis=0),axis=0),interpolation = 'None')
    plt.colorbar()
    plt.axis(False)
    plt.title('Confidence Map')
    
    # Save File
    file = saveDirectory + 'MapSide.' + os.path.basename(ImageName) + '.png'
    plt.savefig(file)
    plt.show()
    
    # Other Figures
    plt.rcParams.update({'font.size': 60})
    plt.figure(figsize=[55,20])
    plt.suptitle(time)
    plt.rcParams.update({'font.size': 50})
    plt.subplot(1,2,1)
    plt.imshow(np.flip(Idsp,axis=0),cmap ='gray')
    plt.axis(False)
    plt.title('Observation')
    plt.subplot(1,2,2)
    plt.imshow(np.flip(np.sum(SEG,axis=0)/91,axis=0),interpolation = 'None')
    plt.colorbar()
    plt.axis(False)
    plt.title('Confidence Map')
    
    # Save File
    file = saveDirectory + 'MapNormSide.' + os.path.basename(ImageName) + '.png'
    plt.savefig(file)
    plt.show()

In [ ]:
# Other Figures
plt.rcParams.update({'font.size': 60})
plt.figure(figsize=[80,20])
plt.suptitle(time)
plt.rcParams.update({'font.size': 50})
plt.subplot(1,3,1)
plt.imshow(np.flip(Idsp,axis=0),cmap ='gray')
plt.axis(False)
plt.title('Observation')
plt.subplot(1,3,2)
plt.imshow(np.flip(np.sum(SEG,axis=0),axis=0),interpolation = 'None')
plt.colorbar()
plt.axis(False)
plt.title('Confidence Map - Full Range')
plt.subplot(1,3,3)
plt.imshow(np.flip(np.sum(SEG[28:],axis=0),axis=0),interpolation = 'None')
plt.colorbar()
plt.axis(False)
plt.title('Confidence Map - Partial Range')
    
# Save File
file = saveDirectory + 'MapSide.' + os.path.basename(ImageName) + '.png'
plt.savefig(file)
plt.show()

# Other Figures
plt.rcParams.update({'font.size': 60})
plt.figure(figsize=[80,20])
plt.suptitle(time)
plt.rcParams.update({'font.size': 50})
plt.subplot(1,3,1)
plt.imshow(np.flip(Idsp,axis=0),cmap ='gray')
plt.axis(False)
plt.title('Observation')
plt.subplot(1,3,2)
plt.imshow(np.flip(np.sum(SEG,axis=0)/91,axis=0),interpolation = 'None')
plt.colorbar()
plt.axis(False)
plt.title('Confidence Map - Full Range')
plt.subplot(1,3,3)
plt.imshow(np.flip(np.sum(SEG[28:],axis=0)/(91-28),axis=0),interpolation = 'None')
plt.colorbar()
plt.axis(False)
plt.title('Confidence Map - Partial Range')
    
# Save File
file = saveDirectory + 'MapNormSide.' + os.path.basename(ImageName) + '.png'
plt.savefig(file)
plt.show()

In [7]:
# Ensure Folder Exists
saveDirectory = '/Figures/ConMaps/NewMaps/'
saveDirectory = ROOT_DIR + saveDirectory

if not os.path.exists(saveDirectory):
    os.makedirs(saveDirectory)

In [ ]:
for ImageName in ImageNames:
    
    # Get File Names
    filename = dataset + ImageName
    ACWEfiles = ImageName.split('/')[0]+'/'+ImageName.split('/')[1]+'/'
    newConMap = glob.glob(newConMapFolder + ACWEfiles + '*' + os.path.basename(ImageName) + '.npz')[0]
    
    # Get File Times
    time = dmt.timeFormat(dmt.timeFromFilenameAIA(ImageName))
    
    # Open AIA Image
    I,Idsp,H = openAIA(filename)
    
    # Open Old Confidence Maps
    H,AH,SEG = acweSaveSeg_v5.openSeg(newConMap)
    
    # Open and save each mask
    fig=plt.figure(layout="constrained",figsize=[30,20])
    gs = GridSpec(4,6,figure=fig)
    plt.rcParams.update({'font.size': 60})
    fig.suptitle(time)
    plt.rcParams.update({'font.size': 50})
    for i in range(5):
        
        # Extract File
        if i == 0:
            j = np.where(AH['BACKGROUND_WEIGHT']==1/10.)[0][0]
            seg = SEG[j]
            seg = acweRestoreScale.upscale(seg,AH)
            ax1 = fig.add_subplot(gs[0:2,0:2])
            ax1.imshow(np.flip(Idsp,axis=0),cmap ='gray')
            ax1.contour(np.flip(seg,axis=0),0,colors='r')
            ax1.axis(False)
            ax1.set_title(r'$\lambda_i/\lambda_o=10$')
        elif i == 1:
            j = np.where(AH['BACKGROUND_WEIGHT']==1/25.)[0][0]
            seg = SEG[j]
            seg = acweRestoreScale.upscale(seg,AH)
            ax2 = fig.add_subplot(gs[0:2,2:4])
            ax2.imshow(np.flip(Idsp,axis=0),cmap ='gray')
            ax2.contour(np.flip(seg,axis=0),0,colors='r')
            ax2.axis(False)
            ax2.set_title(r'$\lambda_i/\lambda_o=25$')
        elif i == 2:
            j = np.where(AH['BACKGROUND_WEIGHT']==1/50.)[0][0]
            seg = SEG[j]
            seg = acweRestoreScale.upscale(seg,AH)
            ax3 = fig.add_subplot(gs[0:2,4:6])
            ax3.imshow(np.flip(Idsp,axis=0),cmap ='gray')
            ax3.contour(np.flip(seg,axis=0),0,colors='r')
            ax3.axis(False)
            ax3.set_title(r'$\lambda_i/\lambda_o=50$')
        elif i == 3:
            j = np.where(AH['BACKGROUND_WEIGHT']==1/75.)[0][0]
            seg = SEG[j]
            seg = acweRestoreScale.upscale(seg,AH)
            j = np.where(AH['BACKGROUND_WEIGHT']==1/50.)[0][0]
            seg = SEG[j]
            seg = acweRestoreScale.upscale(seg,AH)
            ax4 = fig.add_subplot(gs[2:4,0:4])
            ax4.imshow(np.flip(Idsp,axis=0),cmap ='gray')
            ax4.contour(np.flip(seg,axis=0),0,colors='r')
            ax4.axis(False)
            ax4.set_title(r'$\lambda_i/\lambda_o=75$')
        else:
            j = np.where(AH['BACKGROUND_WEIGHT']==1/100.)[0][0]
            seg = SEG[j]
            seg = acweRestoreScale.upscale(seg,AH)
            ax5 = fig.add_subplot(gs[2:4,3:5])
            ax5.imshow(np.flip(Idsp,axis=0),cmap ='gray')
            ax5.contour(np.flip(seg,axis=0),0,colors='r')
            ax5.axis(False)
            ax5.set_title(r'$\lambda_i/\lambda_o=100$')
    file = saveDirectory + 'samples.' + os.path.basename(ImageName) + '.png'
    plt.savefig(file)
    fig.show()
    
    # Other Figures
    plt.rcParams.update({'font.size': 60})
    plt.figure(figsize=[55,20])
    plt.suptitle(time)
    plt.rcParams.update({'font.size': 50})
    plt.subplot(1,2,1)
    plt.imshow(np.flip(Idsp,axis=0),cmap ='gray')
    plt.axis(False)
    plt.title('Observation')
    plt.subplot(1,2,2)
    plt.imshow(np.flip(np.sum(SEG,axis=0),axis=0),interpolation = 'None')
    plt.colorbar()
    plt.axis(False)
    plt.title('Confidence Map')
    
    # Save File
    file = saveDirectory + 'MapSide.' + os.path.basename(ImageName) + '.png'
    plt.savefig(file)
    plt.show()
    
    # Other Figures
    plt.rcParams.update({'font.size': 60})
    plt.figure(figsize=[55,20])
    plt.suptitle(time)
    plt.rcParams.update({'font.size': 50})
    plt.subplot(1,2,1)
    plt.imshow(np.flip(Idsp,axis=0),cmap ='gray')
    plt.axis(False)
    plt.title('Observation')
    plt.subplot(1,2,2)
    plt.imshow(np.flip(np.sum(SEG,axis=0)/91,axis=0),interpolation = 'None')
    plt.colorbar()
    plt.axis(False)
    plt.title('Confidence Map')
    
    # Save File
    file = saveDirectory + 'MapNormSide.' + os.path.basename(ImageName) + '.png'
    plt.savefig(file)
    plt.show()

In [ ]:
# Speical Case

# Get File Names
filename = glob.glob(dataset + '*/*/' + specialCase)[0]
oldConMap = glob.glob(newConMapFolder + '*/*/*' + specialCase + '*')[0]

# Get File Times
time = dmt.timeFormat(dmt.timeFromFilenameAIA(specialCase))

# Open AIA Image
I,Idsp,H = openAIA(filename)

# Open Old Confidence Maps
H,AH,SEG = acweSaveSeg_v5.openSeg(oldConMap)

# Other Figures
plt.rcParams.update({'font.size': 60})
plt.figure(figsize=[80,20])
plt.suptitle(time)
plt.rcParams.update({'font.size': 50})
plt.subplot(1,3,1)
plt.imshow(np.flip(Idsp,axis=0),cmap ='gray')
plt.axis(False)
plt.title('Observation')
plt.subplot(1,3,2)
plt.imshow(np.flip(np.sum(SEG,axis=0),axis=0),interpolation = 'None')
plt.colorbar()
plt.axis(False)
plt.title('Confidence Map - Full Range')
plt.subplot(1,3,3)
plt.imshow(np.flip(np.sum(SEG[28:],axis=0),axis=0),interpolation = 'None')
plt.colorbar()
plt.axis(False)
plt.title('Confidence Map - Partial Range')
    
# Save File
file = saveDirectory + 'MapSide.' + os.path.basename(ImageName) + '.png'
plt.savefig(file)
plt.show()

# Other Figures
plt.rcParams.update({'font.size': 60})
plt.figure(figsize=[80,20])
plt.suptitle(time)
plt.rcParams.update({'font.size': 50})
plt.subplot(1,3,1)
plt.imshow(np.flip(Idsp,axis=0),cmap ='gray')
plt.axis(False)
plt.title('Observation')
plt.subplot(1,3,2)
plt.imshow(np.flip(np.sum(SEG,axis=0)/91,axis=0),interpolation = 'None')
plt.colorbar()
plt.axis(False)
plt.title('Confidence Map - Full Range')
plt.subplot(1,3,3)
plt.imshow(np.flip(np.sum(SEG[28:],axis=0)/(91-28),axis=0),interpolation = 'None')
plt.colorbar()
plt.axis(False)
plt.title('Confidence Map - Partial Range')
    
# Save File
file = saveDirectory + 'MapNormSide.' + os.path.basename(ImageName) + '.png'
plt.savefig(file)
plt.show()

In [ ]:
for ImageName in ImageNames:
    
    # Get File Names
    filename = dataset + ImageName
    ACWEfiles = ImageName.split('/')[0]+'/'+ImageName.split('/')[1]+'/'
    oldConMap = glob.glob(oldConMapFolder + ACWEfiles + '*' + os.path.basename(ImageName) + '.npz')[0]
    newConMap = glob.glob(newConMapFolder + ACWEfiles + '*' + os.path.basename(ImageName) + '.npz')[0]
    
    # Get File Times
    time = dmt.timeFormat(dmt.timeFromFilenameAIA(ImageName))
    
    # Open AIA Image
    I,Idsp,H = openAIA(filename)
    
    # Open Old Confidence Maps
    oH,oAH,oSEG = acweSaveSeg_v5.openSeg(oldConMap)
    nH,nAH,nSEG = acweSaveSeg_v5.openSeg(newConMap)
    
    # Other Figures
    plt.rcParams.update({'font.size': 70})
    plt.figure(figsize=[85,20])
    plt.suptitle(time)
    plt.rcParams.update({'font.size': 60})
    plt.subplot(1,3,1)
    plt.imshow(np.flip(Idsp,axis=0),cmap ='gray')
    plt.axis(False)
    plt.title('Observation')
    plt.subplot(1,3,2)
    plt.imshow(np.flip(np.sum(oSEG,axis=0),axis=0),interpolation = 'None')
    plt.colorbar()
    plt.axis(False)
    plt.title('Independent Segmentations')
    plt.subplot(1,3,3)
    plt.imshow(np.flip(np.sum(nSEG,axis=0),axis=0),interpolation = 'None')
    plt.colorbar()
    plt.axis(False)
    plt.title('Sucessive Evolutions')
    
    # Save File
    file = saveDirectory + 'MapComp.' + os.path.basename(ImageName) + '.png'
    plt.savefig(file)
    plt.show()
    
    # Other Figures
    plt.rcParams.update({'font.size': 70})
    plt.figure(figsize=[85,20])
    plt.suptitle(time)
    plt.rcParams.update({'font.size': 60})
    plt.subplot(1,3,1)
    plt.imshow(np.flip(Idsp,axis=0),cmap ='gray')
    plt.axis(False)
    plt.title('Observation')
    plt.subplot(1,3,2)
    plt.imshow(np.flip(np.sum(oSEG,axis=0)/91,axis=0),interpolation = 'None')
    plt.colorbar()
    plt.axis(False)
    plt.title('Independent Segmentations')
    plt.subplot(1,3,3)
    plt.imshow(np.flip(np.sum(nSEG,axis=0)/91,axis=0),interpolation = 'None')
    plt.colorbar()
    plt.axis(False)
    plt.title('Sucessive Evolutions')
    
    # Save File
    file = saveDirectory + 'MapNormComp.' + os.path.basename(ImageName) + '.png'
    plt.savefig(file)
    plt.show()